In [2]:
import mne
import numpy as np
import os
from pathlib import Path
from scipy import signal
import pickle
from tqdm import tqdm

ModuleNotFoundError: No module named 'mne'

## Load EEG Data

In [ ]:
def load_eeg_file(filepath):
    raw = mne.io.read_raw_edf(filepath, preload=True, verbose=False)
    return raw

def load_subject_data(subject_id, base_path='eegDataset', runs=[1, 2, 3]):
    subject_dir = Path(base_path) / f'S{subject_id:03d}'
    raw_list = []
    target_sfreq = None
    
    for run in runs:
        filepath = subject_dir / f'S{subject_id:03d}R{run:02d}.edf'
        if filepath.exists():
            raw = load_eeg_file(str(filepath))
            
            if target_sfreq is None:
                target_sfreq = raw.info['sfreq']
            elif raw.info['sfreq'] != target_sfreq:
                raw.resample(target_sfreq, verbose=False)
            
            raw_list.append(raw)
    
    if raw_list:
        return mne.concatenate_raws(raw_list, verbose=False)
    return None

## Preprocessing Pipeline

In [ ]:
def preprocess_raw(raw):
    raw.filter(l_freq=1.0, h_freq=40.0, verbose=False)
    raw.set_eeg_reference('average', projection=True, verbose=False)
    raw.apply_proj(verbose=False)
    return raw

def extract_eeg_channels(raw):
    eeg_channels = [ch for ch in raw.ch_names if 'EEG' in ch or ch.startswith(('Fp', 'F', 'C', 'P', 'O', 'T'))]
    if eeg_channels:
        raw.pick_channels(eeg_channels, verbose=False)
    return raw

## Time-Frequency Feature Extraction

In [ ]:
def compute_spectrogram(data, sfreq, window_size=256, overlap=128):
    spectrograms = []
    for channel_data in data:
        f, t, Sxx = signal.spectrogram(channel_data, fs=sfreq, 
                                       nperseg=window_size, 
                                       noverlap=overlap)
        spectrograms.append(Sxx)
    return np.array(spectrograms), f, t

def segment_data(raw, segment_length=4.0, overlap=2.0):
    data = raw.get_data()
    sfreq = raw.info['sfreq']
    
    segment_samples = int(segment_length * sfreq)
    overlap_samples = int(overlap * sfreq)
    step = segment_samples - overlap_samples
    
    segments = []
    for start in range(0, data.shape[1] - segment_samples + 1, step):
        segment = data[:, start:start + segment_samples]
        segments.append(segment)
    
    return segments, sfreq

## Normalize Features

In [ ]:
def normalize_spectrograms(spectrograms):
    spectrograms = np.array(spectrograms)
    spectrograms = np.log10(spectrograms + 1e-10)
    
    mean = np.mean(spectrograms)
    std = np.std(spectrograms)
    spectrograms = (spectrograms - mean) / (std + 1e-8)
    
    return spectrograms, mean, std

## Process All Subjects

In [ ]:
def process_all_subjects(num_subjects=109, base_path='eegDataset', max_segments_per_subject=5):
    all_spectrograms = []
    all_labels = []
    
    for subject_id in tqdm(range(1, num_subjects + 1), desc='Processing subjects'):
        try:
            raw = load_subject_data(subject_id, base_path)
            
            if raw is None:
                continue
            
            raw = extract_eeg_channels(raw)
            raw = preprocess_raw(raw)
            
            segments, sfreq = segment_data(raw, segment_length=4.0, overlap=2.0)
            segments = segments[:max_segments_per_subject]
            
            for segment in segments:
                spec, f, t = compute_spectrogram(segment, sfreq)
                all_spectrograms.append(spec)
                all_labels.append(subject_id - 1)
            
            del raw, segments
            
        except Exception as e:
            print(f"\nWarning: Skipping subject {subject_id}: {e}")
            continue
    
    return np.array(all_spectrograms), np.array(all_labels)

## Execute Preprocessing

In [ ]:
X, y = process_all_subjects(num_subjects=109)
print(f"Data shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Number of unique subjects: {len(np.unique(y))}")

## Normalize and Save

In [ ]:
X_normalized, mean, std = normalize_spectrograms(X)

data = {
    'X': X_normalized,
    'y': y,
    'mean': mean,
    'std': std
}

os.makedirs('preprocessed', exist_ok=True)

with open('preprocessed/preprocessed_eeg_data.pkl', 'wb') as f:
    pickle.dump(data, f)

print("Preprocessing complete. Data saved to 'preprocessed/preprocessed_eeg_data.pkl'")